In [8]:
# Lab 11 MNIST and Convolutional Neural Network
import numpy as np
import tensorflow as tf
import random

random.seed(3)

(x_train,y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255
x_test = x_test /255

print(x_train.shape)
print(x_test.shape)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

learning_rate = 0.001
training_epochs = 12
batch_size = 128

tf.model = tf.keras.Sequential()
# L1
tf.model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
tf.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
# L2
tf.model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
tf.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
# L3 fully connected
tf.model.add(tf.keras.layers.Flatten())
tf.model.add(tf.keras.layers.Dense(units=10, kernel_initializer='glorot_normal', activation='softmax'))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate), metrics=['accuracy'])
tf.model.summary()

tf.model.fit(x_train, y_train, epochs=training_epochs, batch_size=batch_size)

# predict 10 random hand-writing data
y_predicted = tf.model.predict(x_test)
for x in range(0,10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index :", random_index,
         "actual y :", np.argmax(y_test[random_index]),
         "predicted y :", np.argmax(y_predicted[random_index]))

score = tf.model.evaluate(x_test, y_test)
print("loss : ", score[0])
print("accuracy : ", score[1])

(60000, 28, 28)
(10000, 28, 28)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 13, 13, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 5, 5, 32)         0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 800)               0         
                                                                 
 dense_3 (Dense)      

In [11]:
# Lab 12 RNN
import numpy as np
import tensorflow as tf

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello(그 다음 문자를 예측하도록 이렇게 학습)
# x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

num_classes = 5
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

x_one_hot = np.array([[[1, 0, 0, 0, 0],    # h 0
                       [0, 1, 0, 0, 0],    # i 1
                       [1, 0, 0, 0, 0],    # h 0
                       [0, 0, 1, 0, 0],    # e 2
                       [0, 0, 0, 1, 0],    # l 3
                       [0, 0, 0, 1, 0]]],  # l 3
                     dtype=np.float32)

y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print(x_one_hot.shape)
print(y_one_hot.shape)

tf.model = tf.keras.Sequential()

# make cell and add it to RNN layer
# input_shape = (1,6,5) => number of sequence (batch), length of sequence, size of input dim
cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))
tf.model.add(tf.keras.layers.RNN(cell=cell, return_sequences=True))

# single LSTM layer can be used as well instead of creating LSTMCell
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

# fully connected layer
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

# train
tf.model.fit(x_one_hot, y_one_hot, epochs=50)
tf.model.summary()

predictions = tf.model.predict(x_one_hot)
print(predictions)
for i, prediction in enumerate(predictions):
    print(prediction)
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print(result_str)
    print("\tPrediction str: ", ''.join(result_str))

(1, 6, 5)
(1, 6, 5)
Epoch 1/50
1/1 [==============================] - 1s 974ms/step - loss: 1.6460 - accuracy: 0.1667
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: 1.5436 - accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 1.4727 - accuracy: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 1.3881 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 1.2928 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 5ms/step - loss: 1.1857 - accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 1.0636 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.9352 - accuracy: 0.8333
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 0.8102 - accuracy: 0.8333
Epoch 10/50
1/1 [==============================] - 0s 4ms/step - loss: 0.7027 - accuracy: 0.8333
Epoch 11/50
1/1 [

In [ ]:
tf.keras.layers.LSTM
tf.keras.layres.LSTMCell
tf.keras.layers.RNN
tf.keras.layers.TimeDistributed